
<a href="https://colab.research.google.com/github/takzen/pytorch-black-belt/blob/main/37_PyTorch_Lightning_Refactor.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [ ]:
# --------------------------------------------------------------
# ☁️ COLAB SETUP (Automatyczna instalacja środowiska)
# --------------------------------------------------------------
import sys
import os

# Sprawdzamy, czy jesteśmy w Google Colab
if 'google.colab' in sys.modules:
    print('☁️ Wykryto środowisko Google Colab. Konfiguruję...')

    # 1. Pobieramy plik requirements.txt bezpośrednio z repozytorium
    !wget -q https://raw.githubusercontent.com/takzen/ai-engineering-handbook/main/requirements.txt -O requirements.txt

    # 2. Instalujemy biblioteki
    print('⏳ Instaluję zależności (to może chwilę potrwać)...')
    !pip install -q -r requirements.txt

    print('✅ Gotowe! Środowisko jest zgodne z repozytorium.')
else:
    print('💻 Wykryto środowisko lokalne. Zakładam, że masz już uv/venv.')


# 🥋 Lekcja 37: PyTorch Lightning (Koniec ze Spaghetti Code)

PyTorch Lightning (PL) to nie jest nowa biblioteka ML. To **nakładka organizacyjna** na PyTorch.
Wymusza na Tobie podział kodu na logiczne bloki:
1.  **Co to jest model?** (`__init__`)
2.  **Jak to liczy?** (`forward`)
3.  **Jak to trenować?** (`training_step`)
4.  **Jakim optymalizatorem?** (`configure_optimizers`)

Resztę (pętle, urządzenia, logowanie, paski postępu) bierze na siebie klasa **`Trainer`**.

**Zysk:** Twój kod staje się niezależny od sprzętu. Zmieniasz `accelerator="gpu"` na `tpu` jedną linijką, bez przepisywania ani jednego znaku w modelu.

In [10]:
# Instalacja (standard w projektach pro)
# !uv add lightning

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import lightning as L

# Włącza Tensor Cores (TF32) na nowszych GPU NVIDIA (Ampere+).
# Zamienia precyzję FP32 na TF32 dla mnożenia macierzy.
# Efekt: Duże przyspieszenie (nawet 3x) przy pomijalnej utracie dokładności.
torch.set_float32_matmul_precision('medium')

# Konfiguracja
BATCH_SIZE = 64

## Krok 1: LightningModule

Zamiast dziedziczyć po `nn.Module`, dziedziczymy po `L.LightningModule`.
Zauważ, czego **NIE MA** w kodzie poniżej:
*   Nie ma `.to(device)` (PL robi to sam).
*   Nie ma `.backward()` (PL robi to sam).
*   Nie ma `optimizer.step()` i `zero_grad()` (PL robi to sam).

Kod staje się czystą matematyką problemu.

In [7]:
class LitModel(L.LightningModule):
    def __init__(self, input_dim=10, hidden_dim=64, output_dim=1):
        super().__init__()
        # Definicja warstw (tak jak w zwykłym PyTorch)
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
        # Zapisujemy hiperparametry (automatycznie logowane!)
        self.save_hyperparameters()

    def forward(self, x):
        return self.net(x)

    def training_step(self, batch, batch_idx):
        # To jest wnętrze pętli 'for batch in loader'
        x, y = batch
        
        # Forward
        y_hat = self(x)
        
        # Loss
        loss = F.mse_loss(y_hat, y)
        
        # Logowanie (pojawia się na pasku postępu)
        self.log("train_loss", loss, prog_bar=True)
        
        return loss

    def configure_optimizers(self):
        # Definiujemy optymalizator (i ew. scheduler)
        return torch.optim.Adam(self.parameters(), lr=0.001)

print("Model Lightning zdefiniowany.")

Model Lightning zdefiniowany.


## Krok 2: Dane (DataLoader)

PL działa ze standardowymi `DataLoaderami` PyTorcha.
Stwórzmy proste dane.

In [8]:
# Generujemy dane
train_data = torch.randn(1000, 10)
train_targets = torch.randn(1000, 1)

dataset = TensorDataset(train_data, train_targets)
# num_workers=0 dla bezpieczeństwa na Windows w notatniku
train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, num_workers=0)

print("DataLoader gotowy.")

DataLoader gotowy.


## Krok 3: Trainer (Silnik)

Tutaj dzieje się magia. `Trainer` to Twój "Inżynier ML".
Mówisz mu:
*   `max_epochs=5`
*   `accelerator="auto"` (Sam wykryje czy masz GPU, CPU czy MPS).
*   `devices=1`

On sam zajmie się paskiem postępu, obsługą błędów i pętlą.

In [9]:
# Inicjalizacja modelu
model = LitModel()

# Inicjalizacja Trenera
# enable_checkpointing=False dla uproszczenia (żeby nie tworzył folderów w demo)
trainer = L.Trainer(
    max_epochs=5,
    accelerator="auto", # Magia: sam znajdzie GPU
    devices=1,
    enable_checkpointing=False,
    logger=False # Wyłączamy logowanie do plików dla czystości demo
)

print("🚀 Start treningu z Lightning...")
trainer.fit(model=model, train_dataloaders=train_loader)
print("✅ Koniec.")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params | Mode  | FLOPs
----------------------------------------------------
0 | net  | Sequential | 769    | train | 0    
----------------------------------------------------
769       Trainable params
0         Non-trainable params
769       Total params
0.003     Total estimated model params size (MB)
4         Modules in train mode
0         Modules in eval mode
0         Total Flops


🚀 Start treningu z Lightning...
Epoch 4: 100%|██████████| 16/16 [00:00<00:00, 226.29it/s, train_loss=0.608]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 16/16 [00:00<00:00, 223.14it/s, train_loss=0.608]
✅ Koniec.


## 🥋 Black Belt Summary

Dlaczego Lightning to standard w pracy (a nie tylko czysty PyTorch)?

1.  **Czytelność:** Kiedy wchodzisz do nowego projektu, od razu wiesz, gdzie szukać modelu (`__init__`), a gdzie logiki uczenia (`training_step`).
2.  **Skalowalność:** Chcesz uruchomić ten sam kod na 8 GPU? Wystarczy zmienić `devices=8` w Trainerze. W czystym PyTorch musiałbyś użyć `DistributedDataParallel` i przepisać połowę kodu.
3.  **Funkcjonalności:** Mixed Precision (`precision="16-mixed"`), Gradient Clipping, Profiler - wszystko to są flagi w Trainerze.

**Zasada:** Prototypuj w czystym PyTorch (żeby zrozumieć), wdrażaj w Lightning (żeby utrzymać porządek).